In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDetermineBonds
IPythonConsole.ipython_3d = True
import rdkit
rdkit.__version__

In [ ]:
mols = []
names = []
molecules_dir = "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_global_embedding_exp_with_noise/nequip/interactions=3/l=4/position_channels=5/channels=64/global_embed=True/fait=1.0/pit=1.0/step=best/molecules/"
#molecules_dir = "/Users/ameyad/Documents/spherical-harmonic-net/analyses/molecules/downloaded"
for molecules_file in os.listdir(molecules_dir):
    if not molecules_file.endswith(".xyz"):
        continue

    mol = Chem.MolFromXYZFile(os.path.join(molecules_dir, molecules_file))
    mol = Chem.Mol(mol)
    valid_charge = False
    for charge in [0, -1, 1]:
        try:
            rdDetermineBonds.DetermineBonds(mol,charge=charge)
            valid_charge = True
            break
        except ValueError:
            continue
    
    if not valid_charge:
        print(f"Could not find valid charge for {molecules_file}")
        continue
    
    print(f"Found valid charge {charge} for {molecules_file}")

    for bond in mol.GetBonds():
        if bond.GetBondType() == Chem.BondType.DOUBLE:
            bond.SetStereo(Chem.BondStereo.STEREONONE)
        elif bond.GetBondType() == Chem.BondType.SINGLE:
            bond.SetBondDir(Chem.BondDir.NONE)

    # suppl = Chem.ResonanceMolSupplier(conn_mol,Chem.ResonanceFlags.UNCONSTRAINED_ANIONS | Chem.ResonanceFlags.UNCONSTRAINED_CATIONS)
    # for res_mol in suppl:
    #     if res_mol is None:
    #         continue
    #     print(res_mol)
    mols.append(mol)
    names.append(molecules_file)


Draw.MolsToGridImage(mols, molsPerRow=4, legends=names, useSVG=True)        

In [ ]:
def load_molecule_sdf(file):
    mol = Chem.SDMolSupplier(file, removeHs=False)[0]
    if mol is None:
        return None

    positions = mol.GetConformer().GetPositions()
    atom_types = [encoder[atom.GetAtomicNum()] for atom in mol.GetAtoms()]
    bond_orders = rdkit.Chem.rdmolops.GetAdjacencyMatrix(mol, useBO=True)
    return positions, atom_types, None, None, bond_orders

encoder = {
    1: 0,
    5: 1,
    6: 2,
    7: 3,
    8: 4,
}
path = "/Users/ameyad/Documents/spherical-harmonic-net/analyses/analysed_workdirs/qm9_global_embedding_exp_with_noise/nequip/interactions=3/l=4/position_channels=5/channels=64/global_embed=True/fait=1.0/pit=1.0/step=best_old/bonded_and_relaxed_molecules/C6H5_seed=0.sdf"
load_molecule_sdf(path)